In [8]:
import math
import numpy as np
import pandas as pd
from collections import deque

# Import models
from models.bus import Bus, BusSystem
from models.bus_station import BusStation
from models.drone import Drone, DroneFleet
from models.package import Package, get_package_by_id, get_task_by_id
from models.task_optimizer import MultiTaskOptimizer

# Import utilities
from utils.bus_utils import find_nearest_bus_station, find_nearest_bus_stations, calculate_best_line
from utils.distance_utils import calculate_transport_distance
from utils.prediction_utils import (
    predict_bus_arrival_times, 
    predict_single_bus_arrival, 
    optimize_station_selection
)

def run_simulation(task_list, bus_stations_df, total_time=5000):
    """
    Modified version of run_simulation with fixed bus_stations reference.
    
    Args:
        task_list (list): List of task dictionaries
        bus_stations_df (DataFrame): DataFrame of bus stations
        total_time (int, optional): Total simulation time. Defaults to 5000.
        
    Returns:
        tuple: (simulation_records, bus_records, package_records) - Simulation data for analysis
    """
    # Set simulation parameters
    tolerance = 10
    station_rank = 1
    
    # Create bus stations
    bus_stations = [
        BusStation(row['x'], row['y'], row['line'])
        for index, row in bus_stations_df.iterrows()
    ]
    
    # Initialize simulation
    # Pass bus_stations to DroneFleet constructor

    bus_lines = ['Line 1', 'Line 2']
    
    bus_system = BusSystem()
    bus_system.simulate(bus_lines, bus_stations_df)
    
    # Set bus_system in drone_fleet

    
    # Initialize task status and packages
    tasks = task_list.copy()  # Create a copy to avoid modifying original
    packages = []
    
    for task in tasks:
        task['pickup_status'] = 'unassigned'
        task['delivery_status'] = 'unassigned'
        # Use task index as package ID
        package = Package(task['index'], task['index'])
        packages.append(package)
        task['package_id'] = task['index']
    
    # Calculate bus station mapping
    station_mapping = {(station.x, station.y): idx for idx, station in enumerate(bus_stations)}
    drone_fleet = DroneFleet(initial_drone_count=5, bus_stations=bus_stations, packages=packages)
    drone_fleet.bus_system = bus_system
    # Create multi-task optimizer with bus_stations properly passed
    task_optimizer = MultiTaskOptimizer(drone_fleet, bus_system, bus_stations, bus_lines)
    
    # Initialize simulation records
    simulation_records = []
    bus_records = []
    package_records = []
    
    # Start simulation
    current_time = 0
    task_index = 0
    
    
    # Main simulation loop
    while current_time <= total_time:
        # Process task queues
        drone_fleet.process_delivery_task_queue()
        drone_fleet.process_direct_task_queue()
        drone_fleet.process_pickup_task_queue()
        
        # Check for new tasks
        new_tasks_found = False
        while task_index < len(tasks) and tasks[task_index]['pickup_time'] <= current_time:
            task = tasks[task_index]
            if task['pickup_status'] == 'unassigned':
                # Add task to optimizer
                task_optimizer.add_task(task)
                new_tasks_found = True
            
            task_index += 1
        
        # If new tasks found, run optimization
        if new_tasks_found:
            task_optimizer.process_tasks(current_time)
        
        # Process completed pickup tasks that need delivery
        for task in tasks:
            if (
                task['pickup_status'] == 'completed'
                and task['delivery_status'] == 'unassigned'
            ):
                # Check if delivery assignment time has been calculated
                if 'delivery_assign_time' in task:
                    # If time to assign delivery, do so
                    if current_time >= task['delivery_assign_time']:
                        # Use real-time bus prediction to choose best delivery station
                        _, delivery_rank, _ = optimize_station_selection(
                            task, bus_system, bus_stations, current_time
                        )
                        
                        success_delivery = drone_fleet.try_allocate_delivery_task(
                            task, 
                            task['delivery_x'], 
                            task['delivery_y'], 
                            auto_create_drone=False
                        )
                        
                        if not success_delivery:
                            drone_fleet.add_delivery_task_to_queue(
                                task, task['delivery_x'], task['delivery_y']
                            )
                else:
                    # If no delivery_assign_time, predict bus arrival
                    
                    # Find bus transporting the package
                    package = get_package_by_id(task['package_id'], packages)
                    if package and package.status == 'on_bus' and package.bus_id is not None:
                        bus = next((b for b in bus_system.buses if b.bus_id == package.bus_id), None)
                        
                        if bus:
                            # Find nearest delivery stations
                            delivery_stations = find_nearest_bus_stations(
                                task['delivery_x'], task['delivery_y'],
                                bus_stations, task['line'], 3
                            )
                            
                            if delivery_stations:
                                # Predict when bus will arrive at delivery stations
                                best_station = None
                                best_arrival_time = float('inf')
                                
                                for station in delivery_stations:
                                    arrival_time = predict_single_bus_arrival(
                                        bus, (station.x, station.y), current_time
                                    )
                                    
                                    if arrival_time is not None and arrival_time < best_arrival_time:
                                        best_arrival_time = arrival_time
                                        best_station = station
                                
                                if best_station:
                                    # Set delivery task time
                                    task['delivery_assign_time'] = best_arrival_time
                                    task['selected_delivery_station'] = (best_station.x, best_station.y)
                                    print(f"Task {task['index']}: Scheduled delivery at time {best_arrival_time}")
        
        # Move drones and buses
        drone_fleet.move_all_drones()
        bus_system.simulate(bus_lines, bus_stations_df)
        
        # Handle drone-bus interactions
        for drone in drone_fleet.drones:
            # Handle drones waiting for pickup bus
            if (
                drone.status == 'waiting_at_bus_station_for_pickup_bus' and 
                drone.current_task is not None and 
                drone.current_task['pickup_status'] == 'waiting_for_bus'
            ):
                # Use real-time bus prediction to find fastest arriving bus
                current_line = drone.current_task['line']
                
                # Get nearest delivery station
                nearest_delivery_station = find_nearest_bus_station(
                    drone.current_task['delivery_x'], 
                    drone.current_task['delivery_y'], 
                    bus_stations, 
                    current_line
                )
                
                # Get bus arrival predictions
                bus_predictions = predict_bus_arrival_times(
                    bus_system, current_line, (drone.x, drone.y), current_time
                )
                
                # Check if bus is already at station
                bus = bus_system.check_buses_at_station_for_pickup(
                    drone.x, drone.y, 
                    line=current_line, 
                    delivery_station=(nearest_delivery_station.x, nearest_delivery_station.y)
                )
                
                if bus and bus.can_pickup_package():
                    # If bus is at station and can take package
                    package = get_package_by_id(drone.current_task['package_id'], packages)
                    if package:
                        bus.load_package(package)
                        package.bus_id = bus.bus_id
                        package.current_drone_id = None
                        package.update_status('on_bus')
                        drone.current_task['pickup_status'] = 'completed'
                        drone.current_task = None
                        drone.status = 'idle'
                        drone.pickup_tasks_completed += 1
                        
                        # Predict package arrival at delivery station for scheduling
                        if package.task_id is not None:
                            task = get_task_by_id(package.task_id, tasks)
                            if task:
                                # Find nearest delivery station
                                delivery_station = find_nearest_bus_station(
                                    task['delivery_x'], task['delivery_y'], 
                                    bus_stations, current_line
                                )
                                
                                # Predict bus arrival at delivery station
                                arrival_time = predict_single_bus_arrival(
                                    bus, (delivery_station.x, delivery_station.y), current_time
                                )
                                
                                if arrival_time is not None:
                                    # Set delivery task time
                                    task['delivery_assign_time'] = arrival_time
                                    task['selected_delivery_station'] = (delivery_station.x, delivery_station.y)
                                    print(f"Task {task['index']}: Scheduled delivery at time {arrival_time}")
                
                elif not bus and bus_predictions:
                    # If no bus at station but one coming soon, wait
                    next_bus = bus_predictions[0]
                    wait_time = next_bus['wait_time']
                    
                    if wait_time <= 5:  # If wait time is short, print info
                        print(f"Drone {drone.drone_id} waiting for bus {next_bus['bus_id']} arriving in {wait_time:.1f} time units")
            
            # Handle drones waiting for delivery bus
            elif drone.status == 'waiting_at_bus_station_for_delivery':
                package = get_package_by_id(drone.current_task['package_id'], packages)
                
                if package and package.bus_id is not None:
                    # Use real-time prediction to check for approaching bus
                    bus_predictions = predict_bus_arrival_times(
                        bus_system, drone.current_task['line'], (drone.x, drone.y), current_time
                    )
                    
                    # Check if target bus is in predictions
                    target_bus_prediction = next(
                        (pred for pred in bus_predictions if pred['bus_id'] == package.bus_id), 
                        None
                    )
                    
                    # Check if bus is at station
                    bus = bus_system.check_buses_at_station_for_delivery(drone.x, drone.y, package.bus_id)
                    
                    if bus and bus.current_package:
                        package = bus.unload_package()
                        if package:
                            package.update_status('on_delivery_drone')
                            
                            task = get_task_by_id(package.task_id, tasks)
                            if task is not None:
                                drone.assign_delivery_task_to_delivery(
                                    task, task['delivery_x'], task['delivery_y'], task['line']
                                )
                                package.current_drone_id = drone.drone_id
                            else:
                                print(f"Error: Task not found for package_id {package.package_id}")
                    
                    elif target_bus_prediction and target_bus_prediction['wait_time'] <= 5:
                        # If bus arriving soon, print wait info
                        print(f"Drone {drone.drone_id} waiting for bus {package.bus_id} arriving in {target_bus_prediction['wait_time']:.1f} time units")
        
        # Record package states
        for package in packages:
            task = get_task_by_id(package.task_id, tasks)
            if package.status == 'at_pickup_point':
                x, y = task['pickup_x'], task['pickup_y']
            elif package.status == 'on_pickup_drone':
                drone = next((d for d in drone_fleet.drones if d.drone_id == package.current_drone_id), None)
                x, y = (drone.x, drone.y) if drone else (None, None)
            elif package.status == 'on_bus':
                bus = next((b for b in bus_system.buses if b.current_package == package), None)
                x, y = (bus.x, bus.y) if bus else (None, None)
            elif package.status == 'on_delivery_drone':
                drone = next((d for d in drone_fleet.drones if d.drone_id == package.current_drone_id), None)
                x, y = (drone.x, drone.y) if drone else (None, None)
            elif package.status == 'delivered':
                x, y = task['delivery_x'], task['delivery_y']
            elif package.status == 'on_direct_drone':
                drone = next((d for d in drone_fleet.drones if d.drone_id == package.current_drone_id), None)
                x, y = (drone.x, drone.y) if drone else (None, None)
            elif package.status == 'directly_delivered':
                x, y = task['delivery_x'], task['delivery_y']
            else:
                x, y = None, None

            # Record package state
            package_records.append({
                "time": current_time,
                "package_id": package.package_id,
                "task_id": package.task_id,
                "status": package.status,
                "x": x,
                "y": y,
                "bus_id": package.bus_id,
                "drone_id": package.current_drone_id
            })
        
        # Record drone and bus states
        for drone in drone_fleet.drones:
            simulation_records.append({
                "time": current_time,
                "entity": "drone",
                "id": drone.drone_id,
                "x": drone.x,
                "y": drone.y,
                "status": drone.status,
                "battery": drone.battery_left
            })
        
        for bus in bus_system.buses:
            simulation_records.append({
                "time": current_time,
                "entity": "bus",
                "id": bus.bus_id,
                "x": bus.x,
                "y": bus.y,
                "status": bus.status,
                "direction": bus.direction,
                "has_package": bus.current_package is not None
            })
        
        # Record bus station data
        for bus in bus_system.buses:
            # Get current station number
            current_station = station_mapping.get((bus.x, bus.y), None)
            if current_station is not None:
                # Get corresponding station_index and line info
                station_index = next(
                    (station['station_index'] for station in bus.route if 
                     station['x'] == bus.x and station['y'] == bus.y),
                    None
                )
                line = bus.route[0]['line'] if bus.route else None
                
                # Record data
                bus_records.append({
                    "time": current_time,
                    "entity": "bus",
                    "id": bus.bus_id,
                    "station": current_station,
                    "station_index": station_index,
                    "line": line,
                    "has_package": bus.current_package is not None
                })
        
        # Print status every 50 time units
        if current_time % 50 == 0:
            idle_drones = sum(1 for drone in drone_fleet.drones if drone.status == 'idle')
            active_drones = len(drone_fleet.drones) - idle_drones
            packages_on_drones = sum(1 for package in packages if package.status in 
                                   ['on_pickup_drone', 'on_delivery_drone', 'on_direct_drone'])
            packages_on_buses = sum(1 for package in packages if package.status == 'on_bus')
            packages_delivered = sum(1 for package in packages if package.status in 
                                    ['delivered', 'directly_delivered'])
            
            print(f"\nTime {current_time} - System Status:")
            print(f"Active drones: {active_drones}, Idle drones: {idle_drones}")
            print(f"Packages on drones: {packages_on_drones}, On buses: {packages_on_buses}, Delivered: {packages_delivered}")
            print(f"Remaining tasks in optimizer: {len(task_optimizer.pending_tasks)}")
            
            # Print drone battery stats
            low_battery_drones = sum(1 for drone in drone_fleet.drones if drone.battery_left < 50)
            if low_battery_drones > 0:
                print(f"Warning: {low_battery_drones} drones have low battery (<50)")
        
        current_time += 1
    
    # Print final summary
    print("\nSimulation complete. Task and drone summary:")
    for drone in drone_fleet.drones:
        print(f"Drone {drone.drone_id}: Completed {drone.delivery_tasks_completed} delivery tasks and {drone.pickup_tasks_completed} pickup tasks.")
    
    # Calculate completion rate and average delivery time
    completed_tasks = sum(1 for task in tasks if task['delivery_status'] == 'completed')
    completion_rate = completed_tasks / len(tasks) * 100 if tasks else 0
    
    delivery_times = []
    for package in packages:
        if package.status in ['delivered', 'directly_delivered']:
            # Find first and last package records to calculate delivery time
            first_record = next((r for r in package_records if r['package_id'] == package.package_id), None)
            last_record = next((r for r in reversed(package_records) if r['package_id'] == package.package_id 
                               and r['status'] in ['delivered', 'directly_delivered']), None)
            
            if first_record and last_record:
                delivery_time = last_record['time'] - first_record['time']
                delivery_times.append(delivery_time)
    
    avg_delivery_time = sum(delivery_times) / len(delivery_times) if delivery_times else 0
    
    print(f"\nOverall Performance:")
    print(f"Task completion rate: {completion_rate:.2f}%")
    print(f"Average delivery time: {avg_delivery_time:.2f} time units")
    print(f"Total drone count: {len(drone_fleet.drones)}")
    
    # Return simulation data
    return simulation_records, bus_records, package_records



In [9]:
import pandas as pd

# Create simple bus stations data with integer coordinates
# Format: x, y, line, station_index
bus_stations_data = [
    # Line 1 stations
    {'x': 0, 'y': 0, 'line': 'Line 1', 'station_index': 0},
    {'x': 20, 'y': 0, 'line': 'Line 1', 'station_index': 1},
    {'x': 40, 'y': 0, 'line': 'Line 1', 'station_index': 2},
    {'x': 60, 'y': 0, 'line': 'Line 1', 'station_index': 3},
    {'x': 80, 'y': 0, 'line': 'Line 1', 'station_index': 4},
    {'x': 100, 'y': 0, 'line': 'Line 1', 'station_index': 5},
    
    # Line 2 stations
    {'x': 0, 'y': 20, 'line': 'Line 2', 'station_index': 0},
    {'x': 0, 'y': 40, 'line': 'Line 2', 'station_index': 1},
    {'x': 0, 'y': 60, 'line': 'Line 2', 'station_index': 2},
    {'x': 0, 'y': 80, 'line': 'Line 2', 'station_index': 3},
    {'x': 0, 'y': 100, 'line': 'Line 2', 'station_index': 4}
]

# Convert to DataFrame
bus_stations_df = pd.DataFrame(bus_stations_data)

# Create simple task list
# Each task has pickup and delivery locations, time, and assigned bus line
task_list = [
    {
        'index': 0,
        'pickup_x': 10, 'pickup_y': 5,  # Near Line 1
        'delivery_x': 90, 'delivery_y': 5,  # Near Line 1
        'pickup_time': 10,
        'line': 'Line 1'
    },
    {
        'index': 1,
        'pickup_x': 5, 'pickup_y': 15,  # Near intersection of Line 1 and Line 2
        'delivery_x': 5, 'delivery_y': 90,  # Near Line 2
        'pickup_time': 20,
        'line': 'Line 2'
    },
    {
        'index': 2,
        'pickup_x': 30, 'pickup_y': 30,  # Somewhat distant from any line
        'delivery_x': 70, 'delivery_y': 10,  # Near Line 1
        'pickup_time': 30,
        'line': 'Line 1'
    },
    {
        'index': 3,
        'pickup_x': 50, 'pickup_y': 50,  # Far from any line (potential direct delivery)
        'delivery_x': 95, 'delivery_y': 95,  # Far from any line
        'pickup_time': 40,
        'line': None  # Indicates potential direct delivery
    },
    {
        'index': 4,
        'pickup_x': 5, 'pickup_y': 50,  # Near Line 2
        'delivery_x': 50, 'delivery_y': 5,  # Near Line 1
        'pickup_time': 50,
        'line': 'Line 2'
    }
]

# Display the data
print("Bus Stations:")
print(bus_stations_df)
print("\nTask List:")
for task in task_list:
    print(f"Task {task['index']}: Pickup at ({task['pickup_x']}, {task['pickup_y']}) at time {task['pickup_time']}, "
          f"Delivery to ({task['delivery_x']}, {task['delivery_y']}), Line: {task['line']}")

# This is how you would use these in your simulation
# simulation_records, bus_records, package_records = run_simulation(task_list, bus_stations_df)

Bus Stations:
      x    y    line  station_index
0     0    0  Line 1              0
1    20    0  Line 1              1
2    40    0  Line 1              2
3    60    0  Line 1              3
4    80    0  Line 1              4
5   100    0  Line 1              5
6     0   20  Line 2              0
7     0   40  Line 2              1
8     0   60  Line 2              2
9     0   80  Line 2              3
10    0  100  Line 2              4

Task List:
Task 0: Pickup at (10, 5) at time 10, Delivery to (90, 5), Line: Line 1
Task 1: Pickup at (5, 15) at time 20, Delivery to (5, 90), Line: Line 2
Task 2: Pickup at (30, 30) at time 30, Delivery to (70, 10), Line: Line 1
Task 3: Pickup at (50, 50) at time 40, Delivery to (95, 95), Line: None
Task 4: Pickup at (5, 50) at time 50, Delivery to (50, 5), Line: Line 2


In [10]:


# Run simulation
simulation_records, bus_records, package_records = run_simulation(task_list, bus_stations_df)

# Convert records to DataFrames for analysis
simulation_df = pd.DataFrame(simulation_records)
bus_simulation_df = pd.DataFrame(bus_records)
package_simulation_df = pd.DataFrame(package_records)

print("Simulation data saved to DataFrames for analysis")

TypeError: DroneFleet.__init__() got an unexpected keyword argument 'packages'